# PREDECIR PREGUNTAS CERRADAS EN SOES - MODELOS
---


### **Librerias**

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q pandas_read_xml
!pip install -q xlsxwriter #Motor de escritura de archivos xlsx

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas_read_xml as pdx
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
es_stops = set(stopwords.words('spanish'))

### **Rutas a utilizar**

In [ ]:
dataset = '/content/drive/MyDrive/PPC/data'
xlsx = '/content/drive/MyDrive/PPC/xlsx'
csv = '/content/drive/MyDrive/PPC/csv'
pkl = '/content/drive/MyDrive/PPC/pkl'

### **Funciones**

In [ ]:
# def del_labels(content):
#   content = re.sub("<c>", 'cb ', content)
#   content = re.sub("<", '', content)
#   content = re.sub(">", ' ', content)
#   content = re.sub("[++]", 'p', content)
#   return content

In [ ]:
# def prepare(data):
#     data = str(data)
#     data = data.lower() #Conversion a Minusculas
#     data = del_labels(data)
#     return data

### **Archivo Posts.xml**

**Cargar archivo XML**

https://pypi.org/project/pandas-read-xml/
https://colab.research.google.com/github/minchulkim87/pandas_read_xml/blob/master/pandas_read_xml_example.ipynb#scrollTo=ezlRmb4JYfan

In [ ]:
df = pdx.read_xml(dataset + '/Posts.xml',['posts'])
df

,row
0,"[{'@Id': '1', '@PostTypeId': '1', '@AcceptedAn..."


In [ ]:
df = df.pipe(flatten)
df.head()
# 323016

,row
0,"{'@Id': '1', '@PostTypeId': '1', '@AcceptedAns..."
1,"{'@Id': '2', '@PostTypeId': '2', '@ParentId': ..."
2,"{'@Id': '3', '@PostTypeId': '1', '@AcceptedAns..."
3,"{'@Id': '4', '@PostTypeId': '2', '@ParentId': ..."
4,"{'@Id': '5', '@PostTypeId': '1', '@AcceptedAns..."


In [ ]:
df = df.pipe(flatten)
df.head()

,row|@Id,row|@PostTypeId,row|@AcceptedAnswerId,row|@CreationDate,row|@Score,row|@ViewCount,row|@Body,row|@OwnerUserId,row|@LastEditorUserId,row|@LastEditDate,row|@LastActivityDate,row|@Title,row|@Tags,row|@AnswerCount,row|@CommentCount,row|@FavoriteCount,row|@ContentLicense,row|@ParentId,row|@LastEditorDisplayName,row|@OwnerDisplayName,row|@CommunityOwnedDate,row|@ClosedDate
0,1,1,2,2015-10-29T15:56:52.933,34,558,<p>Estoy creando un servicio usando <em>ASP.NE...,23,13558,2019-07-07T21:36:17.737,2021-03-12T17:49:40.687,La creación manual de un alias de ruta con un ...,<.net><asp.net-web-api><asp.net>,1,2,4,CC BY-SA 4.0,NaN,NaN,NaN,NaN,NaN
1,2,2,NaN,2015-10-29T19:14:23.673,27,NaN,<p>He encontrado la solución.</p>\n\n<p>Result...,23,NaN,NaN,2015-10-29T19:14:23.673,NaN,NaN,NaN,2,NaN,CC BY-SA 3.0,1,NaN,NaN,NaN,NaN
2,3,1,9,2015-10-29T23:54:31.947,18,765,"<p>Luego de ver cierto código, me he dado cuen...",21,20,2015-12-03T16:24:56.370,2015-12-18T20:19:25.133,¿Es igual utilizar .AsString que .Text para ob...,<delphi>,3,1,NaN,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN
3,4,2,NaN,2015-10-30T00:45:47.640,5,NaN,"<p><code>.AsString</code> devuelve el mismo ""<...",24,25,2015-12-14T00:58:15.613,2015-12-14T00:58:15.613,NaN,NaN,NaN,0,NaN,CC BY-SA 3.0,3,NaN,NaN,NaN,NaN
4,5,1,208,2015-10-30T01:15:27.267,31,32092,<p>¿Cuál es la forma más eficiente de separar ...,24,729,2016-09-27T17:46:24.900,2020-06-30T02:23:26.197,¿Cómo separar las palabras que contiene un str...,<c++><string>,7,1,3,CC BY-SA 3.0,NaN,NaN,NaN,NaN,NaN


**Fitrado - Eliminación de columnas innecesarias**

https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

Eliminar columnas: 
https://www.analyticslane.com/2019/03/25/como-eliminar-columnas-y-filas-en-un-dataframe-pandas/

In [ ]:
posts = df.drop(['row|@AcceptedAnswerId','row|@CreationDate','row|@LastEditorUserId','row|@LastEditDate','row|@LastActivityDate','row|@ContentLicense','row|@ParentId' ,'row|@LastEditorDisplayName','row|@OwnerDisplayName','row|@CommunityOwnedDate'], axis=1)
posts.head()

,row|@Id,row|@PostTypeId,row|@Score,row|@ViewCount,row|@Body,row|@OwnerUserId,row|@Title,row|@Tags,row|@AnswerCount,row|@CommentCount,row|@FavoriteCount,row|@ClosedDate
0,1,1,34,558,<p>Estoy creando un servicio usando <em>ASP.NE...,23,La creación manual de un alias de ruta con un ...,<.net><asp.net-web-api><asp.net>,1,2,4,NaN
1,2,2,27,NaN,<p>He encontrado la solución.</p>\n\n<p>Result...,23,NaN,NaN,NaN,2,NaN,NaN
2,3,1,18,765,"<p>Luego de ver cierto código, me he dado cuen...",21,¿Es igual utilizar .AsString que .Text para ob...,<delphi>,3,1,NaN,NaN
3,4,2,5,NaN,"<p><code>.AsString</code> devuelve el mismo ""<...",24,NaN,NaN,NaN,0,NaN,NaN
4,5,1,31,32092,<p>¿Cuál es la forma más eficiente de separar ...,24,¿Cómo separar las palabras que contiene un str...,<c++><string>,7,1,3,NaN


**Filtrar por Tipo de Publicación**

In [ ]:
posts = posts[posts['row|@PostTypeId'] == '1']

In [ ]:
posts.head()

,row|@Id,row|@PostTypeId,row|@Score,row|@ViewCount,row|@Body,row|@OwnerUserId,row|@Title,row|@Tags,row|@AnswerCount,row|@CommentCount,row|@FavoriteCount,row|@ClosedDate
0,1,1,34,558,<p>Estoy creando un servicio usando <em>ASP.NE...,23,La creación manual de un alias de ruta con un ...,<.net><asp.net-web-api><asp.net>,1,2,4,NaN
2,3,1,18,765,"<p>Luego de ver cierto código, me he dado cuen...",21,¿Es igual utilizar .AsString que .Text para ob...,<delphi>,3,1,NaN,NaN
4,5,1,31,32092,<p>¿Cuál es la forma más eficiente de separar ...,24,¿Cómo separar las palabras que contiene un str...,<c++><string>,7,1,3,NaN
5,6,1,22,2470,<p>¿Cómo estar seguro o qué pautas seguir a la...,24,"¿Cuándo usar y es seguro emplear const_cast, s...",<c++><casting>,2,0,1,NaN
7,8,1,13,290,<p>Para un proyecto web en el que estoy partic...,23,"""Unknown provider: uiGmapGoogleMapApiProvider""...",<google-maps><angularjs><jasmine><karma-jasmine>,1,1,1,NaN


In [ ]:
posts.shape

(152731, 12)

**Preprocesamiento de Etiquetas**

In [ ]:
# posts['Tags'] = posts.apply(lambda posts: prepare(posts['row|@Tags']), axis=1)

**Filtrar por etiqueta**

In [ ]:
# mask = posts['Tags'].str.contains('cpp|cb')
# posts= posts[mask]

**Eliminacion de PostTypeId**

In [ ]:
posts = posts.drop(['row|@PostTypeId'], axis=1)
posts.head()

,row|@Id,row|@Score,row|@ViewCount,row|@Body,row|@OwnerUserId,row|@Title,row|@Tags,row|@AnswerCount,row|@CommentCount,row|@FavoriteCount,row|@ClosedDate
0,1,34,558,<p>Estoy creando un servicio usando <em>ASP.NE...,23,La creación manual de un alias de ruta con un ...,<.net><asp.net-web-api><asp.net>,1,2,4,NaN
2,3,18,765,"<p>Luego de ver cierto código, me he dado cuen...",21,¿Es igual utilizar .AsString que .Text para ob...,<delphi>,3,1,NaN,NaN
4,5,31,32092,<p>¿Cuál es la forma más eficiente de separar ...,24,¿Cómo separar las palabras que contiene un str...,<c++><string>,7,1,3,NaN
5,6,22,2470,<p>¿Cómo estar seguro o qué pautas seguir a la...,24,"¿Cuándo usar y es seguro emplear const_cast, s...",<c++><casting>,2,0,1,NaN
7,8,13,290,<p>Para un proyecto web en el que estoy partic...,23,"""Unknown provider: uiGmapGoogleMapApiProvider""...",<google-maps><angularjs><jasmine><karma-jasmine>,1,1,1,NaN


In [ ]:
posts.shape

(152731, 11)

**Renombrar columnas**

In [ ]:
posts.columns = ['Id','Score','ViewCount', 'Body','OwnerUserId','Title','AnswerCount','CommentCount','FavoriteCount','ClosedDate','Tags']
posts.columns

Index(['Id', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'Title',
       'AnswerCount', 'CommentCount', 'FavoriteCount', 'ClosedDate', 'Tags'],
      dtype='object')

In [ ]:
posts.head()

,Id,Score,ViewCount,Body,OwnerUserId,Title,AnswerCount,CommentCount,FavoriteCount,ClosedDate,Tags
0,1,34,558,<p>Estoy creando un servicio usando <em>ASP.NE...,23,La creación manual de un alias de ruta con un ...,<.net><asp.net-web-api><asp.net>,1,2,4,NaN
2,3,18,765,"<p>Luego de ver cierto código, me he dado cuen...",21,¿Es igual utilizar .AsString que .Text para ob...,<delphi>,3,1,NaN,NaN
4,5,31,32092,<p>¿Cuál es la forma más eficiente de separar ...,24,¿Cómo separar las palabras que contiene un str...,<c++><string>,7,1,3,NaN
5,6,22,2470,<p>¿Cómo estar seguro o qué pautas seguir a la...,24,"¿Cuándo usar y es seguro emplear const_cast, s...",<c++><casting>,2,0,1,NaN
7,8,13,290,<p>Para un proyecto web en el que estoy partic...,23,"""Unknown provider: uiGmapGoogleMapApiProvider""...",<google-maps><angularjs><jasmine><karma-jasmine>,1,1,1,NaN


**Guardar Filtrado de Posts.xml**

In [ ]:
posts.to_excel(xlsx + '/posts_filter.xlsx', encoding='utf-8',  engine='xlsxwriter', index=False)
posts.to_csv(csv + '/posts_filter.csv', encoding='utf-8', index=False)
posts.to_pickle(pkl + '/posts_filter.pkl')

### **Archivo PostHistory.xml**

**Cargar archivo XML**

In [ ]:
ds = pdx.read_xml(dataset + '/PostHistory.xml',['posthistory'])
ds

KeyboardInterrupt: ignored

In [ ]:
ds = ds.pipe(flatten)
ds.head()

In [ ]:
ds = ds.pipe(flatten)
ds.head()

**Eliminacion de columnas innecesarias PostHistory**

In [ ]:
posth = ds.drop(['row|@RevisionGUID','row|@CreationDate','row|@ContentLicense','row|@UserDisplayName','row|@Id'], axis=1)
posth.head()

In [ ]:
posth.shape

**Filtrado de preguntas cerradas de SOES**

In [ ]:
posth = posth[posth['row|@PostHistoryTypeId'].astype(int) == 10]
posth.head()

In [ ]:
posth.shape

**Eliminacion de PostHistoryTypeId**

In [ ]:
posth = posth.drop(['row|@PostHistoryTypeId'], axis=1)
posth.head()

**Renombrar columnas**

In [ ]:
posth.columns = ['PostId','UserId', 'Text','Comment']
posth.columns

**Guardar Filtrado de PostHistory.xml**

In [ ]:
posth.to_csv(csv + '/posth_filter.csv', encoding='utf-8', index=False)
posth.to_excel(xlsx + '/posth_filter.xlsx', encoding='utf-8',  engine='xlsxwriter', index=False)
posth.to_pickle(pkl + '/posth_filter.pkl')

### **Archivo User.xml**

**Leer archivo xml**

In [ ]:
du = pdx.read_xml(dataset + '/Users.xml',['users'])
du

,row
0,"[{'@Id': '-1', '@Reputation': '1', '@CreationD..."


In [ ]:
du = du.pipe(flatten)

In [ ]:
du = du.pipe(flatten)
du.head()

,row|@Id,row|@Reputation,row|@CreationDate,row|@DisplayName,row|@LastAccessDate,row|@WebsiteUrl,row|@Location,row|@AboutMe,row|@Views,row|@UpVotes,row|@DownVotes,row|@AccountId,row|@ProfileImageUrl
0,-1,1,2015-10-26T21:36:24.767,Comunidad,2015-10-26T21:36:24.767,,en la granja de servidores,"<p>Hola, no soy una persona real.</p>\n\n<p>¡S...",510,9652,18239,-1,NaN
1,1,101,2015-10-26T22:50:12.710,Geoff Dalgas,2021-01-05T19:45:26.807,http://stackoverflow.com,"Corvallis, OR",<p>Dev #2 who helped create Stack Overflow cur...,2887,0,0,2,https://i.stack.imgur.com/nDllk.png?s=128&g=1
2,2,101,2015-10-26T22:51:07.007,Taryn,2020-08-18T17:39:10.693,https://tarynpivots.com,Arizona,"<p>developer, techie, nerd.... DBA at Stack Ov...",91,0,0,188123,https://i.stack.imgur.com/zgBsf.jpg?s=128&g=1
3,3,101,2015-10-26T22:52:12.923,Jon Ericson,2019-11-13T18:17:55.843,https://jlericson.com/,Downtown Burbank,"<p><a href=""https://jlericson.com/2021/04/30/d...",75,0,0,1083,NaN
4,4,101,2015-10-26T23:18:40.547,Juan M,2021-05-26T16:29:20.137,NaN,Behind the Glass,<p>Lover of all things sharp including pencils...,221,56,0,6254215,https://i.stack.imgur.com/7f2Tn.png?s=128&g=1


**Eliminación de columnas innecesarias**

In [ ]:
users = du.drop(['row|@CreationDate','row|@DisplayName','row|@LastAccessDate','row|@WebsiteUrl','row|@Location','row|@AboutMe','row|@Views'	,'row|@UpVotes'	,'row|@DownVotes'	,'row|@AccountId','row|@ProfileImageUrl'], axis=1)
users.head()

,row|@Id,row|@Reputation
0,-1,1
1,1,101
2,2,101
3,3,101
4,4,101


**Renombrar columnas**

In [ ]:
users.columns = ['Id','Reputation']
users.columns

Index(['Id', 'Reputation'], dtype='object')

In [ ]:
users.head()

,Id,Reputation
0,-1,1
1,1,101
2,2,101
3,3,101
4,4,101


**Guardar Filtrado de Users.xml**

In [ ]:
users.to_csv(csv + '/users_filter.csv', encoding='utf-8', index=False)
users.to_excel(xlsx + '/users_filter.xlsx', encoding='utf-8',  engine='xlsxwriter', index=False)
users.to_pickle(pkl + '/users_filter.pkl')

### **Generar dataframe de coincidencias**

In [ ]:
posth = pd.read_excel(xlsx + '/posth_filter.xlsx')
post = pd.read_excel(xlsx + '/posts_filter.xlsx')
user = pd.read_excel(xlsx + '/users_filter.xlsx')

In [ ]:
post.shape

(152731, 11)

In [ ]:
post.head(2)

,Id,Score,ViewCount,Body,OwnerUserId,Title,AnswerCount,CommentCount,FavoriteCount,ClosedDate,Tags
0,1,34,558,<p>Estoy creando un servicio usando <em>ASP.NE...,23.0,La creación manual de un alias de ruta con un ...,<.net><asp.net-web-api><asp.net>,1,2,4.0,NaN
1,3,18,765,"<p>Luego de ver cierto código, me he dado cuen...",21.0,¿Es igual utilizar .AsString que .Text para ob...,<delphi>,3,1,NaN,NaN


In [ ]:
posth.head(2)

,PostId,UserId,Text,Comment
0,35,-1.0,"{""Voters"":[{""Id"":90,""DisplayName"":""fthiella""},...",104
1,175,-1.0,"{""Voters"":[{""Id"":20,""DisplayName"":""Carlos Muño...",105


In [ ]:
user.head(2)

,Id,Reputation
0,-1,1
1,1,101


In [ ]:
posth.shape

(8209, 4)

In [ ]:
user.shape

(212047, 2)

**Coincidencias de ublicaciones(posts) y las preguntas cerradas(posthistory)**

In [ ]:
close_comp = post.loc[post['Id'].isin(posth.PostId)]
close_comp.shape

(8134, 11)

In [ ]:
close_comp.head(2)

,Id,Score,ViewCount,Body,OwnerUserId,Title,AnswerCount,CommentCount,FavoriteCount,ClosedDate,Tags
13,35,33,56041,<p>Debo guardar fechas y horas en diferentes t...,21.0,¿Debo utilizar un campo de tipo DateTime o Tim...,<mysql><datetime><timestamp>,2,4,3.0,NaN
46,175,-8,349,<p><strong>Actualización:</strong></p>\n\n<p>¿...,217.0,Estándar para nombrar setters y getters,<poo><funciones><estándares>,2,9,NaN,2020-07-23T22:32:14.130


**Sustituir los valores nulos con 0**

In [ ]:
close_comp = close_comp.fillna(0)

**Cambiar el formato a entero de columna**

In [ ]:
close_comp.OwnerUserId = close_comp.OwnerUserId.astype(int)
close_comp.OwnerUserId.value_counts()

0         124
38304      26
22584      16
31217      14
10019      13
         ... 
33021       1
186622      1
20735       1
43264       1
10884       1
Name: OwnerUserId, Length: 6396, dtype: int64

**Buscar coincidencias de usuarios que tengan preguntas cerradas**

In [ ]:
user_close = close_comp.loc[close_comp['OwnerUserId'].isin(user.Id)]
user_close = user_close.reset_index(drop=True)
user_close.head(2)

,Id,Score,ViewCount,Body,OwnerUserId,Title,AnswerCount,CommentCount,FavoriteCount,ClosedDate,Tags
0,35,33,56041,<p>Debo guardar fechas y horas en diferentes t...,21,¿Debo utilizar un campo de tipo DateTime o Tim...,<mysql><datetime><timestamp>,2,4,3.0,0
1,175,-8,349,<p><strong>Actualización:</strong></p>\n\n<p>¿...,217,Estándar para nombrar setters y getters,<poo><funciones><estándares>,2,9,0.0,2020-07-23T22:32:14.130


**Generar dataframe de coincidencias y adjuntar su reputación**

In [ ]:
matches_index = ['Id_User', 'Reputation']
matches_df = pd.DataFrame(columns = matches_index) 

for index, row in user_close.iterrows():
  aux_soes_df = user.loc[user['Id'] == row['OwnerUserId']]

  match_lst = aux_soes_df.values.tolist()[0]
  matches_df = matches_df.append(pd.Series(match_lst, index = matches_index), ignore_index=True)

In [ ]:
matches_df.head(5)

,Id_User,Reputation
0,21,23289
1,217,741
2,83,14856
3,25,16587
4,517,7117


**Concatenamos los resultados para unificar a un solo dataset**

In [ ]:
user_close = pd.concat([user_close, matches_df], axis=1)
user_close.head()

,Id,Score,ViewCount,Body,OwnerUserId,Title,AnswerCount,CommentCount,FavoriteCount,ClosedDate,Tags,Id_User,Reputation
0,35,33,56041,<p>Debo guardar fechas y horas en diferentes t...,21,¿Debo utilizar un campo de tipo DateTime o Tim...,<mysql><datetime><timestamp>,2,4,3.0,0,21,23289
1,175,-8,349,<p><strong>Actualización:</strong></p>\n\n<p>¿...,217,Estándar para nombrar setters y getters,<poo><funciones><estándares>,2,9,0.0,2020-07-23T22:32:14.130,217,741
2,222,9,147,"<p>Hasta hace poco utilizaba Bash. Allí, puede...",83,¿Cómo eliminar el contenido de la consola desd...,<bash><zsh>,1,5,1.0,0,83,14856
3,352,44,2379,"<p>En la empresa que trabajo, si para resolver...",25,¿Qué consideraciones debería tener en cuenta a...,<licencias><código-abierto>,4,3,12.0,0,25,16587
4,415,4,14848,<p>¿Cómo creo una expresión regular que me val...,517,RegEx para validar numeros de movil españoles,<java><string><regex>,2,1,0.0,2015-12-04T17:13:23.443,517,7117


In [ ]:
user_close.shape

(8010, 13)

**Guardar archivo final**

In [ ]:
user_close.to_excel(xlsx + '/closed_question.xlsx', encoding='utf-8',  engine='xlsxwriter', index=False)
user_close.to_csv(csv + '/closed_question.csv', encoding='utf-8', index=False)
user_close.to_pickle(pkl + '/closed_question.pkl')

**Una vez obtenido las preguntas cerradas se tendrá que buscar de manera manual la razón por la cual se cerró la pregunta.**